In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
from sentence_transformers import SentenceTransformer

# Der erste Download des Models dauert einen längeren Augenblick, da einige GB an Daten heruntergeladen werden!
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [ ]:
import numpy as np
vector_function = lambda text: model.encode([text], show_progress_bar=False)[0]

In [ ]:
from qdrant_client import QdrantClient

# man Qdrant auch als Dockercontainer laufen lassen
#qdrant = QdrantClient("http://localhost:6333")

# Erzeugt einen Vektorstorage im Arbeitsspeicher 
# (geht verloren wenn die Variable neu initialisiert wird oder der Notebook Kernel beendet wird)
qdrant = QdrantClient(":memory:")

In [ ]:
from qdrant_client.http.models import VectorParams, PointStruct

collection_name = "the_target_collection"
vector_params = VectorParams(size=768, distance="Cosine")

In [ ]:
# Create the collection
qdrant.create_collection(
    collection_name=collection_name,
    vectors_config=vector_params
)

In [ ]:
from bs4 import BeautifulSoup
import requests

def crawl_website(url: str) -> str:
    crawl_response = requests.get(url)
    if crawl_response.status_code != 200:
        logger.warning(f"Website wurde nicht erfoglreich heruntergeladen - {crawl_response.text}")
        raise ValueError("Website nicht geladen")

    website_text = crawl_response.text
    cleaned_text = BeautifulSoup(website_text).get_text()    
    return cleaned_text

In [ ]:
text1 = crawl_website("https://de.wikipedia.org/wiki/Hulk_Hogan")
text2 = crawl_website("https://de.wikipedia.org/wiki/The_Undertaker")
text3 = crawl_website("https://de.wikipedia.org/wiki/Dwayne_Johnson")

In [ ]:
import itertools

def chunker(text: str) -> list[str]:
    return [''.join(chunk) for chunk in itertools.batched(text, 250)]

data_points = [PointStruct(id=idx, vector=vector_function(c), payload={"text": c}) 
               for idx, c in enumerate(chunker(text=text1) + chunker(text=text2) + chunker(text=text3))]

In [ ]:
qdrant.upsert(points=data_points, collection_name=collection_name)

In [ ]:
query_vector = vector_function("Wann starb Hulk Hogan")

In [ ]:
# Wir kriegen zwar ein Ergebnis ABER wir haben jetzt das Problem das wir nicht wissen zu welchen ursprünglichen Dokument eine Chunk gehört.
# Vector Datenbank arbeiten in der Regel auf Chunk Level. Das heißt, wer "ursprüngliche" Dokumente für einen/den besten Chunk abrufen will, 
# der muss mitverfolgen welche Chunk-ID zu welchem Document gehört. Dieses Mapping muss während dem Chunking aufgebaut werden.
results = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=10
)

matching_chunks = [result.payload for result in results]

### Überarbeitet Version die den Dokumenten Ursprung mitverfolgt

### Anfügen des Antwortdokumentes an eine ChatGPT Interaktion

In [ ]:
from dotenv import load_dotenv
# lädt die (versteckte) Datei die im selben Ordner liegt wie dieses notebook file
# die Datei muss .env benannt sein, diese Dateien sind in der Regel ausgeblendet, 
# weil Dateien mit . andeuten, dass diese im Explorer ausgeblendet werden.
# In der Datei
load_dotenv()

import os
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.environ["API_KEY"]}",
}

import pprint
import requests

def call_openai(payload: dict) -> dict:
    ENDPOINT = "https://api.openai.com/v1/chat/completions"
        
    response = requests.post(ENDPOINT, headers=headers, json=payload)
    if response.status_code != 200:
        logger.warning(f"Fehlerhafte Antwort: {response.text}")
    endpoint_response = response.json()
    return endpoint_response

In [ ]:
payload = {
  "model": "gpt-4o-mini",
  "messages": [
    {"role": "system", 
     "content": "Du bist ein Wrestling Experte der alle Fragen über amerikanische Wrestler der 90er Jahre beantwortet."
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"""Wann ist Hulk Hogan gestorben?
                      Berücksichige folgende Informationen {matching_chunks}
                      """
        }
      ]
    }
  ],
  "temperature": 0.5
}

In [ ]:
endpoint_response = call_openai(payload=payload)
chat_answer = endpoint_response.get('choices', [])[0].get('message', {}).get('content', "FEHLER")
chat_answer